# Dataset 4: Target Segmentation Analysis
## 세그먼트별 겨울 선크림 시장 기회 분석

**분석 목표:**
- 6개 세그먼트(20/30/40대 × 남성/여성)별 선크림과 스포츠 키워드 관심도 비교
- 블루오션 세그먼트 발굴: 높은 스포츠 관심 + 낮은 선크림 인식
- 타겟 페르소나 정의 및 마케팅 우선순위 도출

**데이터:**
- 기간: 2023-01 ~ 2025-11 (35개월)
- 키워드: 선크림, 스키장, 스키, 스노우보드
- 세그먼트: 6개 (남성 20/30/40대, 여성 20/30/40대)

## 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.patches import Patch
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.style.use('default')
sns.set_palette("husl")

print("라이브러리 임포트 완료")

## 2. 데이터 로드 및 전처리

In [ ]:
# 통합 데이터 로드
df = pd.read_csv('04_세그먼트별_통합_데이터.csv')

# 날짜 형식 변환
df['date'] = pd.to_datetime(df['date'])

# 데이터 구조 확인
print("=== 데이터 기본 정보 ===")
print(f"전체 행 수: {len(df):,}")
print(f"기간: {df['date'].min()} ~ {df['date'].max()}")
print(f"\n세그먼트: {df['segment'].unique()}")
print(f"키워드: {df['keyword'].unique()}")
print(f"\n데이터 샘플:")
print(df.head(10))

## 3. 세그먼트별 평균 검색량 분석

In [ ]:
# 세그먼트별 키워드 평균 검색량 계산
segment_avg = df.groupby(['segment', 'keyword'])['search_volume'].mean().reset_index()
segment_pivot = segment_avg.pivot(index='segment', columns='keyword', values='search_volume')

# 정렬: 선크림 검색량 기준 내림차순
segment_pivot = segment_pivot.sort_values('선크림', ascending=False)

print("=== 세그먼트별 키워드 평균 검색량 ===")
print(segment_pivot.round(2))
print("\n상위 3개 세그먼트 (선크림 검색량 기준):")
print(segment_pivot['선크림'].head(3))

## 4. 키워드별 세그먼트 순위 시각화

In [ ]:
# 4개 키워드별 세그먼트 순위
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
keywords = ['선크림', '스키장', '스키', '스노우보드']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']

for idx, keyword in enumerate(keywords):
    ax = axes[idx // 2, idx % 2]
    data = segment_pivot[keyword].sort_values(ascending=True)
    
    bars = ax.barh(data.index, data.values, color=colors[idx], alpha=0.8, edgecolor='black')
    
    # 값 레이블 추가
    for i, (val, segment) in enumerate(zip(data.values, data.index)):
        ax.text(val + 0.5, i, f'{val:.1f}', va='center', fontsize=10, fontweight='bold')
    
    ax.set_xlabel('Average Search Volume', fontsize=12, fontweight='bold')
    ax.set_title(f'{keyword} - Segment Ranking', fontsize=14, fontweight='bold', pad=15)
    ax.grid(axis='x', alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)

plt.tight_layout()
plt.savefig('segment_keyword_ranking.png', dpi=300, bbox_inches='tight')
plt.show()

print("세그먼트별 키워드 순위 시각화 완료")

## 5. 선크림 vs 스포츠 통합 관심도 분석

In [ ]:
# 평균 매트릭스 파일 로드 (더 정확한 데이터)
df_stats = pd.read_csv('04_세그먼트_키워드_평균_매트릭스.csv')

# 컬럼명 확인 및 통일
if 'segment' in df_stats.columns:
    df_stats = df_stats.rename(columns={'segment': '세그먼트'})

print("=== 세그먼트별 평균 검색량 (매트릭스 파일) ===")
print(df_stats)

# 선크림 평균 (기준선)
suncream_avg = df_stats['선크림'].mean()

# 겨울스포츠 통합: 스키장 + 스키 + 스노우보드 평균
df_stats['겨울스포츠'] = df_stats[['스키장', '스키', '스노우보드']].mean(axis=1)
winter_sport_avg = df_stats['겨울스포츠'].mean()

print(f"\n선크림 전체 평균: {suncream_avg:.2f}")
print(f"겨울스포츠 전체 평균: {winter_sport_avg:.2f}")

## 6. 4사분면 분류 및 블루오션 발굴

In [ ]:
# 4사분면 분류
def classify_quadrant(row):
    sc = row['선크림']
    ws = row['겨울스포츠']
    
    if sc >= suncream_avg and ws >= winter_sport_avg:
        return 'A (둘 다 높음)'
    elif sc < suncream_avg and ws >= winter_sport_avg:
        return 'B (블루오션!)'
    elif sc >= suncream_avg and ws < winter_sport_avg:
        return 'C (선크림만)'
    else:
        return 'D (둘 다 낮음)'

df_stats['사분면'] = df_stats.apply(classify_quadrant, axis=1)

print("=" * 60)
print("4사분면 분류 결과")
print("=" * 60)
print(df_stats[['세그먼트', '선크림', '겨울스포츠', '사분면']].to_string(index=False))

# 블루오션 세그먼트 추출
blueocean = df_stats[df_stats['사분면'] == 'B (블루오션!)'].copy()
blueocean = blueocean.sort_values('겨울스포츠', ascending=False)

print("\n" + "=" * 60)
print("💎 블루오션 세그먼트 (선크림↓ + 겨울스포츠↑)")
print("=" * 60)

if len(blueocean) > 0:
    print(f"\n발견: {len(blueocean)}개 블루오션 세그먼트\n")
    for idx, row in blueocean.iterrows():
        print(f"🎯 {row['세그먼트']}")
        print(f"   선크림 인식: {row['선크림']:.1f} (평균 {suncream_avg:.1f} 이하)")
        print(f"   겨울스포츠 관심: {row['겨울스포츠']:.1f} (평균 {winter_sport_avg:.1f} 이상)")
        print(f"   스키장: {row['스키장']:.1f} / 스키: {row['스키']:.1f} / 보드: {row['스노우보드']:.1f}")
        print()
else:
    print("\n⚠️  명확한 블루오션 세그먼트가 발견되지 않았습니다.")
    print("→ 기준을 조정하거나 다른 전략을 고려해야 합니다.")

## 7. 4사분면 시각화

In [ ]:
# 4사분면 시각화
fig, ax = plt.subplots(figsize=(12, 8))

# 색상 매핑
color_map = {
    'A (둘 다 높음)': '#3498db',      # 파랑
    'B (블루오션!)': '#27ae60',       # 초록 (강조)
    'C (선크림만)': '#95a5a6',        # 회색
    'D (둘 다 낮음)': '#ecf0f1'       # 연회색
}

colors = df_stats['사분면'].map(color_map)

# 산점도
scatter = ax.scatter(df_stats['선크림'], df_stats['겨울스포츠'], 
                    c=colors, s=500, alpha=0.8, edgecolors='black', linewidth=2)

# 세그먼트 레이블
for idx, row in df_stats.iterrows():
    weight = 'bold' if row['사분면'] == 'B (블루오션!)' else 'normal'
    ax.annotate(row['세그먼트'], (row['선크림'], row['겨울스포츠']), 
               fontsize=11, ha='center', va='center', fontweight=weight)

# 평균선 (4사분면 구분)
ax.axhline(y=winter_sport_avg, color='red', linestyle='--', alpha=0.6, linewidth=2)
ax.axvline(x=suncream_avg, color='red', linestyle='--', alpha=0.6, linewidth=2)

# 사분면 라벨
offset_x = (df_stats['선크림'].max() - df_stats['선크림'].min()) * 0.05
offset_y = (df_stats['겨울스포츠'].max() - df_stats['겨울스포츠'].min()) * 0.05

ax.text(suncream_avg + offset_x, winter_sport_avg + offset_y, 
       'A\n(둘 다 높음)\n리마인더', fontsize=11, alpha=0.6, ha='left', va='bottom')
ax.text(suncream_avg - offset_x, winter_sport_avg + offset_y, 
       'B\n(블루오션!)\n교육 캠페인', fontsize=12, alpha=0.8, ha='right', va='bottom', 
       color='green', fontweight='bold')
ax.text(suncream_avg + offset_x, winter_sport_avg - offset_y, 
       'C\n(선크림만)\n온라인', fontsize=11, alpha=0.6, ha='left', va='top')
ax.text(suncream_avg - offset_x, winter_sport_avg - offset_y, 
       'D\n(둘 다 낮음)\n우선순위↓', fontsize=11, alpha=0.6, ha='right', va='top')

ax.set_xlabel('Sunscreen Average Search Volume', fontsize=13, fontweight='bold')
ax.set_ylabel('Winter Sports Average Search Volume (Ski Resort+Ski+Snowboard)', fontsize=13, fontweight='bold')
ax.set_title('Target Segment 4-Quadrant Analysis\n(Green = Education Campaign Priority)', 
            fontsize=15, fontweight='bold', pad=20)
ax.grid(alpha=0.3)

# 범례
legend_elements = [
    Patch(facecolor='#27ae60', label='B: Blue Ocean (Top Priority)'),
    Patch(facecolor='#3498db', label='A: Secondary Target'),
    Patch(facecolor='#95a5a6', label='C: Online Target'),
    Patch(facecolor='#ecf0f1', label='D: Low Priority')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=11)

plt.tight_layout()
plt.savefig('quadrant_sports_sunscreen.png', dpi=300, bbox_inches='tight')
plt.show()

print("스포츠 통합 4사분면 분석 완료")

## 8. 히트맵 시각화

In [ ]:
# 세그먼트 × 키워드 히트맵
fig, ax = plt.subplots(figsize=(12, 8))

# 데이터 정규화 (0-100 스케일)
heatmap_data = segment_pivot[['선크림', '스키장', '스키', '스노우보드']].copy()
heatmap_normalized = (heatmap_data - heatmap_data.min().min()) / (heatmap_data.max().max() - heatmap_data.min().min()) * 100

sns.heatmap(heatmap_normalized, annot=heatmap_data.round(1), fmt='g', cmap='YlOrRd', 
            cbar_kws={'label': 'Normalized Score (0-100)'}, linewidths=2, linecolor='white',
            ax=ax, vmin=0, vmax=100)

ax.set_title('Segment × Keyword Heatmap (Search Volume)', fontsize=16, fontweight='bold', pad=20)
ax.set_xlabel('Keyword', fontsize=13, fontweight='bold')
ax.set_ylabel('Segment', fontsize=13, fontweight='bold')
ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=11)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=11)

plt.tight_layout()
plt.savefig('segment_keyword_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("히트맵 시각화 완료")

## 9. 타겟 페르소나 도출

In [ ]:
# 스포츠 통합 지수 계산 (segment_pivot에 추가)
segment_pivot['Sports_Interest'] = segment_pivot[['스키장', '스키', '스노우보드']].mean(axis=1)
segment_pivot['Sunscreen_Interest'] = segment_pivot['선크림']
segment_pivot['Interest_Gap'] = segment_pivot['Sports_Interest'] - segment_pivot['Sunscreen_Interest']
segment_pivot['Gap_Ratio'] = (segment_pivot['Sunscreen_Interest'] / segment_pivot['Sports_Interest'] * 100).round(1)

# 블루오션 세그먼트 우선순위 계산
segment_pivot['Priority_Score'] = (
    segment_pivot['Interest_Gap'] * 0.6 +  # Gap이 큰 세그먼트
    segment_pivot['Sports_Interest'] * 0.4  # 스포츠 관심도 높은 세그먼트
)

priority_ranking = segment_pivot[[
    'Sunscreen_Interest', 'Sports_Interest', 'Interest_Gap', 'Gap_Ratio', 'Priority_Score'
]].sort_values('Priority_Score', ascending=False)

print("=== 타겟 세그먼트 우선순위 ===")
print(priority_ranking)

print("\n=== TOP 3 타겟 페르소나 ===")
for i, (segment, row) in enumerate(priority_ranking.head(3).iterrows(), 1):
    age_group = segment.split('.')[1]
    gender = 'Female' if segment.startswith('f') else 'Male'
    
    print(f"\n[{i}순위] {segment}")
    print(f"  - 성별/연령: {gender} {age_group}대")
    print(f"  - 스포츠 관심도: {row['Sports_Interest']:.1f}")
    print(f"  - 선크림 인식: {row['Sunscreen_Interest']:.1f}")
    print(f"  - 인식 Gap: {row['Interest_Gap']:.1f} (기회 지수)")
    print(f"  - 선크림/스포츠 비율: {row['Gap_Ratio']:.1f}%")
    print(f"  - 우선순위 점수: {row['Priority_Score']:.2f}")

## 10. 시계열 트렌드 분석 (월별)

In [ ]:
# TOP 3 세그먼트 시계열 트렌드
top3_segments = priority_ranking.head(3).index.tolist()

fig, axes = plt.subplots(2, 2, figsize=(18, 12))
keywords_trend = ['선크림', '스키장', '스키', '스노우보드']

for idx, keyword in enumerate(keywords_trend):
    ax = axes[idx // 2, idx % 2]
    
    for segment in top3_segments:
        segment_data = df[(df['segment'] == segment) & (df['keyword'] == keyword)]
        segment_data = segment_data.sort_values('date')
        
        ax.plot(segment_data['date'], segment_data['search_volume'], 
                marker='o', linewidth=2, markersize=4, label=segment, alpha=0.8)
    
    ax.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax.set_ylabel('Search Volume', fontsize=12, fontweight='bold')
    ax.set_title(f'{keyword} - Monthly Trend (TOP 3 Segments)', fontsize=14, fontweight='bold', pad=15)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_axisbelow(True)

plt.tight_layout()
plt.savefig('top3_segments_trend.png', dpi=300, bbox_inches='tight')
plt.show()

print("TOP 3 세그먼트 시계열 트렌드 분석 완료")

## 11. 마케팅 인사이트 정리

In [ ]:
print("=== Dataset 4 핵심 인사이트 ===")
print("\n[1] 블루오션 세그먼트 발굴")
print(f"   - 최우선 타겟: {priority_ranking.index[0]}")
print(f"   - Interest Gap: {priority_ranking.iloc[0]['Interest_Gap']:.1f} (스포츠 관심 대비 선크림 인식 부족)")
print(f"   - 시장 기회: 높은 스포츠 활동 참여 + 낮은 자외선 보호 인식")

print("\n[2] 세그먼트별 차이")
highest_sunscreen = segment_pivot['선크림'].idxmax()
lowest_sunscreen = segment_pivot['선크림'].idxmin()
print(f"   - 선크림 관심 최고: {highest_sunscreen} ({segment_pivot.loc[highest_sunscreen, '선크림']:.1f})")
print(f"   - 선크림 관심 최저: {lowest_sunscreen} ({segment_pivot.loc[lowest_sunscreen, '선크림']:.1f})")
print(f"   - 세그먼트 간 격차: {segment_pivot['선크림'].max() - segment_pivot['선크림'].min():.1f}")

print("\n[3] 스포츠별 타겟 매칭")
for sport in ['스키장', '스키', '스노우보드']:
    top_segment = segment_pivot[sport].idxmax()
    print(f"   - {sport} 최고 관심: {top_segment} ({segment_pivot.loc[top_segment, sport]:.1f})")

print("\n[4] 마케팅 전략 제안")
print("   ① 우선 타겟: TOP 3 블루오션 세그먼트 집중 공략")
print("   ② 메시지: 스키장 자외선 위험성 + Dataset 3 과학적 근거 활용")
print("   ③ 채널: 스포츠별 맞춤 채널 (스키장/스키/스노우보드 커뮤니티)")
print("   ④ 타이밍: 겨울 스포츠 시즌 집중 (12월~2월)")

print("\n[5] Dataset 1-3 연계")
print("   - Dataset 1: 겨울 선크림 시장 존재 (29.0 검색 지수)")
print("   - Dataset 2: 연간 550만 명 스키장 이용자, 고소득층 타겟")
print("   - Dataset 3: 스키장 UV 위험도 79% vs 인식 43% = 36%p Gap")
print("   - Dataset 4: 세그먼트별 Gap 최대화 타겟 발굴 완료")

## 12. 결과 저장

In [ ]:
# 분석 결과 CSV 저장
df_stats.to_csv('04_세그먼트_4사분면_결과.csv', encoding='utf-8-sig', index=False)
segment_pivot.to_csv('04_세그먼트_분석_결과.csv', encoding='utf-8-sig')
priority_ranking.to_csv('04_타겟_우선순위.csv', encoding='utf-8-sig')

print("분석 결과 저장 완료:")
print("  - 04_세그먼트_4사분면_결과.csv (블루오션 분류)")
print("  - 04_세그먼트_분석_결과.csv")
print("  - 04_타겟_우선순위.csv")
print("\n시각화 파일:")
print("  - segment_keyword_ranking.png")
print("  - quadrant_sports_sunscreen.png (통합 4사분면)")
print("  - segment_keyword_heatmap.png")
print("  - top3_segments_trend.png")

## 분석 완료

**Dataset 4 주요 결과:**
1. 블루오션 세그먼트 발굴: 높은 스포츠 관심 + 낮은 선크림 인식
2. 통합 4사분면 분석: 스포츠 3개 통합 지수 vs 선크림
3. TOP 3 타겟 페르소나 정의
4. 마케팅 전략 제안

**다음 단계:**
- Dataset 1-4 통합 리포트 작성
- 최종 발표 자료 준비